In [1]:
import datetime
%load_ext tensorboard

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers

In [2]:
dataframe = pd.read_csv('ageC.csv')

In [3]:
#preprocess here
#Drop un-used columns.
dataframe = dataframe.drop(columns=['fnlwgt', 'class', 'ageC', 'capital-gain', 'capital-loss', 
                                   'capital-lossB'])
dataframe.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,target,capital-gainB
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,40,United-States,1,0
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,50,United-States,1,0
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,40,United-States,0,0
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,40,United-States,0,1
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,30,United-States,1,0


In [4]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [5]:
feature_columns = []

# numeric cols
for header in ['age', 'education-num', 'capital-gainB', 'hours-per-week']:
  feature_columns.append(feature_column.numeric_column(header))

# indicator_columns
indicator_column_names = ['workclass', 'occupation', 'marital-status',
                          'race', 'sex', 'native-country']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [6]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(dataframe, test_size=0.2)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

print(len(train))

35166


In [7]:
checkpoint_path = "census/cp-{epoch:04d}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=0, 
    save_weights_only=True,
    save_freq=1000)


model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)
])


model.compile(optimizer='RMSprop',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1098/1099 [============================>.] - ETA: 0s - loss: 0.4961 - accuracy: 0.7481WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'workclass': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'education': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=string>, 'education-num': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=int64>, 'marital-status': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=string>, 'occupation': <tf.Tensor 'ExpandDims_7:0' shape=(None, 1) dtype=string>, 'relationship': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dtype=string>, 'race': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'sex': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'hours-per-week': <tf.

Epoch 35/100
1099/1099 [==============================] - 1s 994us/step - loss: 0.3211 - accuracy: 0.8446 - val_loss: 0.3259 - val_accuracy: 0.8347
Epoch 36/100
1099/1099 [==============================] - 1s 1ms/step - loss: 0.3126 - accuracy: 0.8499 - val_loss: 0.3253 - val_accuracy: 0.8355
Epoch 37/100
1099/1099 [==============================] - 1s 984us/step - loss: 0.3166 - accuracy: 0.8441 - val_loss: 0.3254 - val_accuracy: 0.8362
Epoch 38/100
1099/1099 [==============================] - 1s 987us/step - loss: 0.3129 - accuracy: 0.8482 - val_loss: 0.3257 - val_accuracy: 0.8488
Epoch 39/100
1099/1099 [==============================] - 1s 979us/step - loss: 0.3134 - accuracy: 0.8494 - val_loss: 0.3263 - val_accuracy: 0.8466
Epoch 40/100
1099/1099 [==============================] - 1s 980us/step - loss: 0.3153 - accuracy: 0.8482 - val_loss: 0.3292 - val_accuracy: 0.8237
Epoch 41/100
1099/1099 [==============================] - 1s 985us/step - loss: 0.3133 - accuracy: 0.8496 - val_lo

Epoch 91/100
1099/1099 [==============================] - 1s 1ms/step - loss: 0.3115 - accuracy: 0.8514 - val_loss: 0.3272 - val_accuracy: 0.8295
Epoch 92/100
1099/1099 [==============================] - 1s 998us/step - loss: 0.3139 - accuracy: 0.8466 - val_loss: 0.3252 - val_accuracy: 0.8457
Epoch 93/100
1099/1099 [==============================] - 1s 1ms/step - loss: 0.3136 - accuracy: 0.8495 - val_loss: 0.3275 - val_accuracy: 0.8386
Epoch 94/100
1099/1099 [==============================] - 1s 987us/step - loss: 0.3120 - accuracy: 0.8488 - val_loss: 0.3275 - val_accuracy: 0.8425
Epoch 95/100
1099/1099 [==============================] - 1s 998us/step - loss: 0.3091 - accuracy: 0.8510 - val_loss: 0.3266 - val_accuracy: 0.8392
Epoch 96/100
1099/1099 [==============================] - 1s 1ms/step - loss: 0.3139 - accuracy: 0.8481 - val_loss: 0.3281 - val_accuracy: 0.8389
Epoch 97/100
1099/1099 [==============================] - 1s 1ms/step - loss: 0.3162 - accuracy: 0.8469 - val_loss: 0.

In [8]:
%tensorboard --logdir logs/fit

Launching TensorBoard...